<a href="https://colab.research.google.com/github/aknsntn/visualisation/blob/main/Data_Tables_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Для выполнения заданий возьмите данные из папки [data](https://drive.google.com/drive/folders/17E4vB5kWr5FNjLKRU4XGoJSVTxn1jkg4?usp=sharing) о продажах. Данные распределены по 4 папкам:
* sales_train.csv - данные о продажах с января 2013 по октябрь 2015.
** date - дата продажы
** date_block_num - номер месяца по порядку следования
** shop_id - идентификатор магазина
** item_id - идентификатор товара
** item_price - цена товара
** item_cnt_day - количество проданного товара одного вида в день

* items.csv - подробная информация о товарах
** item_id - идентификатор товара
** item_name - название товара
** category_id - идентификатор категории товара

* item_categories.csv - подробная информация о категориях товаров
** item_category_id - идентификатор категории товара
** category_name - название категории товара

* shops.csv - подробная информация о магазинах
** shop_id - идентификатор магазина
** shop_name - название магазина


In [ ]:
install.packages("tidyverse")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
library(tidyverse)

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.8
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



# Задание 1

In [ ]:
sales <- read_csv('sample_data/sales_train.csv', col_types = cols(date = col_character(), date_block_num = col_double(), shop_id = col_double(), item_id = col_double(), item_price = col_double(), item_cnt_day = col_double()))

In [ ]:
head(sales)

date,date_block_num,shop_id,item_id,item_price,item_cnt_day
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
02.01.2013,0,59,22154,999.00,1
03.01.2013,0,25,2552,899.00,1
05.01.2013,0,25,2552,899.00,-1
06.01.2013,0,25,2554,1709.05,1
15.01.2013,0,25,2555,1099.00,1
10.01.2013,0,25,2564,349.00,1


1. Найдите топ-10 самых часто покупаемых товаров (не забудьте учесть столбец item_cnt_day).


In [ ]:
top <- sales[c("item_id", "item_cnt_day")]
top <- top %>% group_by(item_id) %>% summarise(item_cnt_day = sum(item_cnt_day)) %>% arrange(desc(item_cnt_day))
head(top, n = 10)

item_id,item_cnt_day
<dbl>,<dbl>
20949,180174
2808,17173
3732,16529
17717,14818
5822,13995
3734,11586
6675,10285
1855,10024
3731,9624


2. Оставьте только те из них, которые покупали все три года (2013, 2014, 2015).

In [ ]:
full_top <- merge(head(top, n = 10), sales, by = "item_id")
full_top <- full_top[ , -7]
names(full_top)[2] <- "item_cnt_day"
top_2013 <- full_top %>% filter(date_block_num < 12)
top_2013 <- unique(top_2013[1])
top_2014 <- full_top %>% filter(date_block_num > 11) %>% filter(date_block_num < 24)
top_2014 <- unique(top_2014[1])
top_2015 <- filter(full_top, date_block_num > 23)
top_2015 <- unique(top_2015[1])
needed_id <- merge(top_2015, merge(top_2013, top_2014, by = "item_id"), by = "item_id")
needed_id

item_id
<dbl>
1855
2808
3732
3734
5822
6675
16787
17717
20949


# Задание 2

In [ ]:
item_categories <- read_csv("sample_data/item_categories.csv")
items <- read_csv("sample_data/items.csv")
items

Rows: 84 Columns: 2
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (1): item_category_name
dbl (1): item_category_id

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 22170 Columns: 3
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (1): item_name
dbl (2): item_id, category_id

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


item_name,item_id,category_id
<chr>,<dbl>,<dbl>
!! IN THE POWER OF HAPPINESS (PLAST) D,0,40
"! ABBYY FineReader 12 Professional Edition Full [PC, Digital Version]",1,76
*** IN THE GLORY OF THE GLORY (UNV) D,2,40
*** BLUE WAVE (Univ) D,3,40
*** BOX (GLASS) D,4,40
*** NEW AMERICAN GRAPHICS (UNI) D,5,40
*** BLOCK ON THE GATES (UNI) D,6,40
*** SHOCK AT DOOR-2 (UNI) D,7,40
*** TEA WITH MUSSOLINI D,8,40


In [ ]:
top_ctg <- merge(sales, items, by = "item_id")

In [ ]:
top_ctg <- top_ctg %>% group_by(category_id) %>% summarise(item_cnt_day = sum(item_cnt_day)) %>% arrange(item_cnt_day)


1. Определите категорию товаров, в которой совершается минимальное число покупок. Выведите айди и название категории.

In [ ]:
min_ctg <- top_ctg[c(1:2),]
min_ctg

category_id,item_cnt_day
<dbl>,<dbl>
10,1
51,1


2. Определите категорию товаров, в которой совершается максимальное число покупок. Выведите айди и название категории.

In [ ]:
top_ctg <- top_ctg %>% group_by(category_id) %>% summarise(item_cnt_day = sum(item_cnt_day)) %>% arrange(desc(item_cnt_day))
max_ctg <- top_ctg[1,]
max_ctg

category_id,item_cnt_day
<dbl>,<dbl>
40,619945


# Задание 3

1. Для категорий из задания 2 найдите среднее количество продаж каждого товара категории в 5 магазинах с наибольшими продажами.

In [ ]:
items_ctg <- merge(items, sales, by = "item_id")

,item_id,item_name,category_id,date,date_block_num,shop_id,item_price,item_cnt_day
,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
1,0,!! IN THE POWER OF HAPPINESS (PLAST) D,40,01.09.2014,20,54,58,1
2,1,"! ABBYY FineReader 12 Professional Edition Full [PC, Digital Version]",76,02.04.2014,15,55,4490,1
3,1,"! ABBYY FineReader 12 Professional Edition Full [PC, Digital Version]",76,04.08.2014,19,55,4490,1


In [ ]:
top_shops <- head(sales %>% group_by(shop_id) %>% summarise(item_cnt_day = sum(item_cnt_day)) %>% arrange(desc(item_cnt_day)), n = 5)
top_shops <- merge(top_shops, items_ctg, by = "shop_id")
top_shops <- top_shops[ , -2]
names(top_shops)[8] <- "item_cnt_day"

In [ ]:
top_shops_max <- merge(top_shops, max_ctg, by = "category_id")
top_shops_max <- top_shops_max[ , -9]
names(top_shops_max)[8] <- "item_cnt_day"
top_shops_max %>% group_by(item_id) %>% summarise(item_cnt_day = mean(item_cnt_day))

item_id,item_cnt_day
<dbl>,<dbl>
0,1.000000
2,1.000000
3,1.000000
4,1.000000
5,1.000000
6,1.000000
7,1.000000
8,1.000000
9,1.000000


In [ ]:
top_shops_min <- merge(top_shops, min_ctg, by = "category_id")
top_shops_min #once again they don't sell this stuff in cool shops

category_id,shop_id,item_id,item_name,date,date_block_num,item_price,item_cnt_day.x,item_cnt_day.y
<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>


2. Найдите месяцы с минимальными и максимальными продажами для каждого товара из пункта 1.


In [ ]:
min_sales <- top_shops_max %>% group_by(item_id, date_block_num) %>% summarise(item_cnt_day = min(item_cnt_day))
min_sales[ , -3]

`summarise()` has grouped output by 'item_id'. You can override using the
`.groups` argument.


item_id,date_block_num
<dbl>,<dbl>
0,20
2,19
2,22
3,18
3,19
4,20
5,23
6,18
7,23


In [ ]:
max_sales <- top_shops_max %>% group_by(item_id, date_block_num) %>% summarise(item_cnt_day = max(item_cnt_day))
max_sales[ , -3]

`summarise()` has grouped output by 'item_id'. You can override using the
`.groups` argument.


item_id,date_block_num
<dbl>,<dbl>
0,20
2,19
2,22
3,18
3,19
4,20
5,23
6,18
7,23


# Задание 4

1. Для категорий из задания 2 найдите среднюю и медианную цену по категории в пределах каждого месяца.


In [ ]:
#mean price for the least popular categories
merge(items_ctg, min_ctg, by = "category_id") %>% group_by(category_id, date_block_num) %>% summarise(item_price = mean(item_price))

`summarise()` has grouped output by 'category_id'. You can override using the
`.groups` argument.


category_id,date_block_num,item_price
<dbl>,<dbl>,<dbl>
10,3,1998
51,18,129


In [ ]:
#mean price for the most popular category
merge(items_ctg, max_ctg, by = "category_id") %>% group_by(category_id, date_block_num) %>% summarise(item_price = mean(item_price))

`summarise()` has grouped output by 'category_id'. You can override using the
`.groups` argument.


category_id,date_block_num,item_price
<dbl>,<dbl>,<dbl>
40,0,244.4859
40,1,252.1749
40,2,260.6450
40,3,254.8524
40,4,247.6276
40,5,245.8609
40,6,236.8121
40,7,234.4702
40,8,241.8186


In [ ]:
#median price for the least popular categories
merge(items_ctg, min_ctg, by = "category_id") %>% group_by(category_id, date_block_num) %>% summarise(item_price = median(item_price))

`summarise()` has grouped output by 'category_id'. You can override using the
`.groups` argument.


category_id,date_block_num,item_price
<dbl>,<dbl>,<dbl>
10,3,1998
51,18,129


In [ ]:
#median price for the most popular category
merge(items_ctg, max_ctg, by = "category_id") %>% group_by(category_id, date_block_num) %>% summarise(item_price = median(item_price))

`summarise()` has grouped output by 'category_id'. You can override using the
`.groups` argument.


category_id,date_block_num,item_price
<dbl>,<dbl>,<dbl>
40,0,199.00
40,1,203.00
40,2,231.00
40,3,199.00
40,4,199.00
40,5,199.00
40,6,149.00
40,7,149.00
40,8,198.00


2. В каждом году найдите месяцы, в которых средняя цена меньше, чем в предыдущем и следующем месяце.

In [ ]:
months <- items_ctg %>% group_by(date_block_num) %>% summarise(item_price = mean(item_price))
months[months] #not fifnished tho..

date_block_num,item_price
<dbl>,<dbl>
0,710.6221
1,695.8669
2,694.6881
3,653.6338
4,624.1800
5,630.8936
6,618.5782
7,624.5769
8,756.2288
